必要なライブラリー読み込み   

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline
%precision 5
import scipy.spatial.distance as dis
import sklearn
import csv
import pprint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
reg = linear_model.LinearRegression()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

必要なデータの読み込み

In [2]:
#rating_data : userId, movieId, rating
rating_data = pd.read_csv("ratings.csv",encoding = "UTF-8") 
#movie_data : movieId, title, genres
movie_data = pd.read_csv("movies.csv",encoding='cp932')
#tag_data : movieId, tagId, relevance, tag
tag_id = pd.read_csv("genome-scores.csv")
tag_detail = pd.read_csv("genome-tags.csv") 
tag_data = pd.merge(tag_id,tag_detail,how="left",on ="tagId")
tag_rel = tag_id['relevance'].to_numpy().reshape([-1,1128])
#tagを持つmovieのmovieIdのリスト
movieId_list = set(tag_data["movieId"])
movieId_list = list(movieId_list)
movieId_list.sort()
#tag_relevance(tagとそれぞれの映画の関係性)
data_relevance = pd.DataFrame(np.array(tag_data["relevance"]).reshape(len(movieId_list),1128))
movieId_tag = pd.DataFrame(movieId_list)
movieId_tag.columns  = ["movieId"]
listed = list(range(1,1129))
data_relevance = data_relevance.rename(columns = lambda x:listed[x])
data_relevance = data_relevance.add_prefix("tag")
tag_relevance = pd.concat([movieId_tag,data_relevance],axis = 1)
#映画のジャンル
list_genres = ["Action","Adventure","Animation","Children","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir",
               "Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western","(no genres listed)"]

userId1の基本データ

In [3]:
data1 = rating_data[rating_data["userId"] == 1]
data1_list = np.array(data1["movieId"])
print("user1のratingに総数、平均、標準偏差")
print(data1["rating"].describe())

flag = movie_data["movieId"].isin(data1_list)
data1_movie = movie_data[flag]

data1_merge= pd.merge(data1,data1_movie,how="left",on ="movieId")
data1_merge

print("             ")
print("user1がどういったジャンルを高く評価しているか")

for i in range(len(list_genres)):
    data1_alpha = data1_merge[data1_merge["genres"].str.contains(list_genres[i])]
    print(list_genres[i],":",data1_alpha["rating"].mean())


user1のratingに総数、平均、標準偏差
count    70.000000
mean      3.814286
std       1.004235
min       0.500000
25%       3.500000
50%       4.000000
75%       4.500000
max       5.000000
Name: rating, dtype: float64
             
user1がどういったジャンルを高く評価しているか
Action : 4.125
Adventure : 3.727272727272727
Animation : 4.0
Children : 3.8333333333333335
Comedy : 3.869565217391304
Crime : 4.0625
Documentary : 2.0
Drama : 3.8679245283018866
Fantasy : 3.6
Film-Noir : 3.5
Horror : 3.5
Musical : 3.7
Mystery : 3.5
Romance : 4.166666666666667
Sci-Fi : 3.7
Thriller : 4.3
War : 3.9
Western : 2.5
(no genres listed) : nan


C:\Users\wataru\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


tagに対しての重回帰分析

In [4]:
#あるuserに関する重回帰分析の関数(引数に求めたいuserIdを入力)
def calc_r_tagrel(x):
    try:
        data = rating_data[rating_data["userId"] == x]
        flag = tag_relevance["movieId"].isin(data["movieId"]) 
        user_tag = tag_relevance[flag]
        user_tag.reset_index(inplace=True)
        user_tag = user_tag.drop("index",axis = 1)
        data_linear_model = pd.merge(data,user_tag)
        X = data_linear_model.drop(["rating","userId","movieId","timestamp",],axis = 1)
        y = data_linear_model["rating"]
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5 , random_state = 0)
        model = LinearRegression()
        model.fit(X_train,y_train)
        print("決定係数(train):{:.3f}".format(model.score(X_train,y_train)))
        print("決定係数(test):{:.3f}".format(model.score(X_test,y_test)))
        print("\n回帰係数\n{}".format(pd.Series(model.coef_,index = X.columns)))
        print("切片：{:.3f}".format(model.intercept_))
        print(model.predict(X_test))
        print("下記にテストデータのratingの予測を記す。")
        print(dict(y_test))
    except:
        print("userIdを入力してください。")

In [5]:
calc_r_tagrel(1)

決定係数(train):1.000
決定係数(test):0.053

回帰係数
tag1       0.001946
tag2      -0.000731
tag3       0.084166
tag4      -0.052203
tag5      -0.048374
             ...   
tag1124   -0.026823
tag1125    0.127559
tag1126   -0.006203
tag1127    0.001644
tag1128    0.001163
Length: 1128, dtype: float64
切片：3.498
[4.35045 3.97576 3.84973 3.90411 3.45025 3.18373 4.47496 3.77762 4.59192
 3.62196 3.2933  5.42448 3.82384 3.43661 4.05862 3.54895 3.46829 3.76209
 4.44249 3.99876 3.67846 3.2177  4.05254 5.03166 3.09081 4.27075 4.69563
 3.72211 3.58275 3.31512 3.95862 3.91073 4.53953 3.14403]
下記にテストデータのratingの予測を記す。
{48: 5.0, 27: 3.0, 32: 2.0, 22: 5.0, 31: 0.5, 45: 2.0, 30: 4.0, 60: 4.0, 55: 3.5, 62: 4.0, 28: 4.0, 41: 5.0, 67: 5.0, 58: 4.0, 52: 2.5, 40: 3.5, 4: 3.5, 51: 3.0, 26: 5.0, 2: 5.0, 66: 3.5, 11: 4.0, 3: 5.0, 42: 3.5, 10: 3.5, 34: 4.0, 61: 3.0, 43: 4.0, 54: 4.5, 53: 2.5, 7: 3.5, 14: 2.5, 33: 5.0, 63: 3.0}


ある1つの映画に対して似ている映画を求める    
ユークリッド距離を用いて、movieIdがxの似ている映画上位y位までを求める。

In [12]:
user_tag_rel = np.array(tag_data[tag_data["movieId"] == 1]["relevance"])
euclid_dis = np.apply_along_axis(lambda x:dis.euclidean(user_tag_rel,x),1,tag_rel)
"""
sort_index_user = np.argsort(euclid_dis)
for i in sort_index_user[:10]:
    flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
    print(movie_data["title"][np.where(flag == True)[0][0]])
"""
print(euclid_dis)

flag = tag_data["movieId"].isin([1])
rel0 = tag_data[flag]["relevance"]
sim = np.apply_along_axis(lambda x:dis.cosine(rel0,x),1,tag_rel)
#sort_index = np.argsort(sim)
"""
for i in sort_index[:10]:
    flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
    print(movie_data["title"][np.where(flag == True)[0][0]])
"""
print(sim)

[0.      5.48894 6.39882 ... 6.08321 6.24865 6.35956]
[0.      0.22385 0.30982 ... 0.28469 0.27625 0.30882]


In [6]:
def euclid_dis_top(x,y):
    try:
        user_tag_rel = np.array(tag_data[tag_data["movieId"] == x]["relevance"])
        euclid_dis = np.apply_along_axis(lambda x:dis.euclidean(user_tag_rel,x),1,tag_rel)
        sort_index_user = np.argsort(euclid_dis)
        for i in sort_index_user[:y]:
            flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
            print(movie_data["title"][np.where(flag == True)[0][0]])
    except:
        print("movieId_list内のmovieIdを引数としてください。")

In [7]:
euclid_dis_top(122914,7)

Avengers: Infinity War - Part II (2019)
Avengers: Infinity War - Part I (2018)
Spider-Man: Far from Home (2019)
Thor: Ragnarok (2017)
Spider-Man: Into the Spider-Verse (2018)
Untitled Spider-Man Reboot (2017)
Black Panther (2017)


cos類似度を用いて、movieIdがxの似ている映画を上位y位まで求める。

In [8]:
def cos_top(x,y):
    try:
        flag = tag_data["movieId"].isin([x])
        rel0 = tag_data[flag]["relevance"]
        sim = np.apply_along_axis(lambda x:dis.cosine(rel0,x),1,tag_rel)
        sort_index = np.argsort(sim)
        for i in sort_index[:y]:
            flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
            print(movie_data["title"][np.where(flag == True)[0][0]])
    except:
        print("movieId_list内のmovieIdを引数としてください。")

In [9]:
cos_top(122914,7)

Avengers: Infinity War - Part II (2019)
Avengers: Infinity War - Part I (2018)
Thor: Ragnarok (2017)
Spider-Man: Far from Home (2019)
Untitled Spider-Man Reboot (2017)
Spider-Man: Into the Spider-Verse (2018)
Black Panther (2017)


movieIdがxの物に対するお勧めをユークリッド距離とcos類似度を用いて求める(1<y<51)

In [7]:
def featured_movie(x,y):
    flag = tag_data["movieId"].isin([x])
    rel0 = tag_data[flag]["relevance"]
    sim = np.apply_along_axis(lambda x:dis.cosine(rel0,x),1,tag_rel)
    sort_index = np.array(tag_id["movieId"][((np.argsort(sim))*1128)[:50]])
    rel_cos = tag_data[tag_data["movieId"].isin(sort_index)]["relevance"].to_numpy().reshape([-1,1128])
    rel_cos_id = np.array(movie_data[movie_data["movieId"].isin(sort_index)]["movieId"])
    euclid_dis = np.apply_along_axis(lambda x:dis.euclidean(rel0,x),1,rel_cos)
    rel_cos_id = rel_cos_id[np.argsort(euclid_dis)]
    for i in rel_cos_id[:y]:
        flag = movie_data["movieId"].isin([i])
        print(movie_data["title"][np.where(flag == True)[0][0]])

In [8]:
featured_movie(122914,10)

Avengers: Infinity War - Part II (2019)
Avengers: Infinity War - Part I (2018)
Spider-Man: Far from Home (2019)
Thor: Ragnarok (2017)
Spider-Man: Into the Spider-Verse (2018)
Untitled Spider-Man Reboot (2017)
Black Panther (2017)
Deadpool 2 (2018)
Shazam! (2019)
Aquaman (2018)


複数の映画に対して似ている映画を求める(featured_movie_clu(x,z)求めたいuserIdをx,求めたいお勧めの数をzとする。)

In [12]:
def featured_movie_clu(x,z):
    try:
        cluster_num = len(rating_data[rating_data["userId"] == x])//50 + 3
        data = rating_data[rating_data["userId"] == x] 
        X = tag_relevance[tag_relevance["movieId"].isin(data["movieId"]) ].drop("movieId",axis = 1)
        Kmeans = KMeans(n_clusters = cluster_num,init ="random",random_state = 0)
        y = Kmeans.fit_predict(X)
        for i in range(cluster_num):
            rel0 = X[y == i].mean()
            sim = np.apply_along_axis(lambda x:dis.cosine(rel0,x),1,tag_rel)
            sort_index = np.array(tag_id["movieId"][((np.argsort(sim))*1128)[:50]])
            rel_cos = tag_data[tag_data["movieId"].isin(sort_index)]["relevance"].to_numpy().reshape([-1,1128])
            rel_cos_id = np.array(movie_data[movie_data["movieId"].isin(sort_index)]["movieId"])
            euclid_dis = np.apply_along_axis(lambda x:dis.euclidean(rel0,x),1,rel_cos)
            rel_cos_id = rel_cos_id[np.argsort(euclid_dis)]
            print("##################クラスターNo.",i, "のおすすめ##################")
            for i in rel_cos_id[:z]:
                flag = movie_data["movieId"].isin([i])
                print(movie_data["title"][np.where(flag == True)[0][0]])
    except ValueError as error:
        print("movieId_list内にあるuserIdを入力してください。")

In [13]:
featured_movie_clu(1,10)

##################クラスターNo. 0 のおすすめ##################
Cold War (2018)
Climax (2018)
The Meyerowitz Stories (2017)
Phantom Thread (2017)
Burning (2018)
Mandy (2018)
First Reformed (2017)
Suspiria (2017)
Midsommar (2019)
Thoroughbreds (2018)
##################クラスターNo. 1 のおすすめ##################
Blue Planet II (2017)
Black Mirror
The Favourite
Isle of Dogs (2018)
Eighth Grade (2018)
Three Billboards Outside Ebbing, Missouri (2017)
BlacKkKlansman (2018)
Making a Murderer (2015)
Joker (2019)
Once Upon a Time in Hollywood (2019)
##################クラスターNo. 2 のおすすめ##################
Winter Light (Nattvardsgﾃ､sterna) (1963)
Strada, La (1954)
Saraband (2003)
Roma (2018)
Fanny and Alexander (Fanny och Alexander) (1982)
Dolce Vita, La (1960)
Wild Strawberries (Smultronstﾃ､llet) (1957)
Three Colors: Blue (Trois couleurs: Bleu) (1993)
Through a Glass Darkly (Sﾃ･som i en spegel) (1961)
Three Colors: Red (Trois couleurs: Rouge) (1994)
##################クラスターNo. 3 のおすすめ##################
Bumblebee (2018)

重み付き平均を用いた総解析    
関数weighted_recommend(x)はuserIdを入力すると、そのuserのクラスター毎のお勧めを算出するものである。

In [10]:
def print_cluster(p):
    cluster_num = len(rating_data[rating_data["userId"] == p])//50 + 3
    data = rating_data[rating_data["userId"] == p] 
    X = tag_relevance[tag_relevance["movieId"].isin(data["movieId"])]
    X = X.drop("movieId",axis = 1)
    Kmeans = KMeans(n_clusters = cluster_num,init ="random",random_state = 0)
    y = Kmeans.fit_predict(X)
    flag = movie_data["movieId"].isin(tag_relevance[tag_relevance["movieId"].isin(data["movieId"])]["movieId"])
    name_print = movie_data[flag]
    print("userId" + str(p) + "のクラスタの個数は" + str(cluster_num) + "であり、0以上" + str(cluster_num - 1) +
          "以下で入力してください。")
    num = int(input("求めたいクラスタ番号を半角で入力してください:"))
    print(name_print["title"][y == num])

In [11]:
print_cluster(1)

userId1のクラスタの個数は4であり、0以上3以下で入力してください。
求めたいクラスタ番号を半角で入力してください:3
1061                                 Dirty Dancing (1987)
1922                    Back to the Future Part II (1989)
1923                   Back to the Future Part III (1990)
2071                        NeverEnding Story, The (1984)
5840        Lord of the Rings: The Two Towers, The (2002)
6258                                  Finding Nemo (2003)
6416    Pirates of the Caribbean: The Curse of the Bla...
7734                                       Shrek 2 (2004)
9122                                        Taxi 2 (2000)
Name: title, dtype: object


In [14]:
def weighted_recommend(x):
    try:
        cluster_num = len(rating_data[rating_data["userId"] == x])//50 + 3
        data = rating_data[rating_data["userId"] == x]
        tag_rel_1 = tag_relevance[tag_relevance["movieId"].isin(data["movieId"]) ]
        X = (tag_rel_1.copy()).drop("movieId",axis = 1)
        Kmeans = KMeans(n_clusters = cluster_num,init ="random",random_state = 0)
        y = Kmeans.fit_predict(X)
        def cal_weighted_sum(v,r):
            center = v.mean(axis = 0)
            m = r.mean()
            sn = np.sign(r-m).reshape(-1,1)
            weigth = ((r-m)**2).reshape(-1,1)
            vector = (v-center)
            return center + (sn*weigth*vector).sum(axis = 0)/weigth.sum()
        for i in range(cluster_num):
            v = np.array(X[y == i])
            r = np.array(data[data["movieId"].isin(tag_rel_1[y==i]["movieId"])]["rating"])
            euclid_rel = np.apply_along_axis(lambda x:dis.euclidean(cal_weighted_sum(v,r),x),1,tag_rel)
            sort_euclid_rel = np.argsort(euclid_rel)
            print("##################クラスターNo.",i, "のおすすめ##################")
            for i in sort_euclid_rel[:5]:
                flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
                print(movie_data["title"][np.where(flag == True)[0][0]])
    except:
        print("movieId_list内にあるuserIdを入力してください。")

In [15]:
weighted_recommend(1)

##################クラスターNo. 0 のおすすめ##################
Cold War (2018)
Faces, Places (2017)
And the Ship Sails On (E la nave va) (1983)
The Meyerowitz Stories (2017)
Burning (2018)
##################クラスターNo. 1 のおすすめ##################
Run Lola Run (Lola rennt) (1998)
Blue Planet II (2017)
Eighth Grade (2018)
Three Billboards Outside Ebbing, Missouri (2017)
Once Upon a Time in Hollywood (2019)
##################クラスターNo. 2 のおすすめ##################
Strada, La (1954)
Three Colors: Blue (Trois couleurs: Bleu) (1993)
Dolce Vita, La (1960)
Three Colors: Red (Trois couleurs: Rouge) (1994)
3-Iron (Bin-jip) (2004)
##################クラスターNo. 3 のおすすめ##################
How to Train Your Dragon: The Hidden World (2019)
Incredibles 2 (2018)
Coco (2017)
How to Train Your Dragon (2010)
Toy Story 4 (2019)


In [ ]:
#LastResearch2より引用
ur_name = input("あなたのuser名をアルファベットのみで入力してください:")

#頻出度数が高い上位300本の映画
rating_counts = rating_data["movieId"].value_counts()
rating_counts = pd.DataFrame(rating_counts)
rating_counts = rating_counts.reset_index()
rating_counts = pd.DataFrame(rating_counts["index"])
rating_counts.columns = ["movieId"]
flag = rating_counts["movieId"].isin(movieId_list)
chosed_data = rating_counts[flag]
chosed_data = chosed_data["movieId"]
chosed_data = np.array(chosed_data[:200])

watch_data = []
while True:
    ur_rating = 0
    #人気上位200位の中から無作為に評価する映画を抽出
    movieId_random = np.random.choice(chosed_data, 1, replace=True)
    ur_movieId = int(movieId_random)
    flag = movie_data["movieId"].isin([ur_movieId])
    print(movie_data["title"][np.where(flag == True)[0][0]])
    ur_rating = input("この映画に対する評価を半角数字0以上5以下で入力してください:")    
    try:
        if 0 <= float(ur_rating) <= 5:
            print("正しい値が入力されました。次の映画の評価をしてください。")
            watch_data.append(int(ur_movieId))
            watch_data.append(float(ur_rating))
            #break
    except:
        if ur_rating =="non":
            print("次の映画の評価に映ります。")
        else:
            print("正しくない値が入力されました。")
    if len(watch_data) == 20:
        print("これで評価は終わりです。お疲れ様でした。")
        break
#評価し終わった映画をデータフレームに移行
watch_movieId =watch_data[::2]
watch_movieId = pd.DataFrame(watch_movieId)
watch_rating = watch_data[1::2]
watch_rating = pd.DataFrame(watch_rating)
ur_recommend = pd.concat([watch_movieId,watch_rating],axis = 1)
ur_recommend.columns = ["movieId","rating"]

cluster_num = len(watch_data)//50 + 3
flag = tag_relevance["movieId"].isin(ur_recommend["movieId"]) 
tag_rel_1 = tag_relevance[flag]
X = tag_rel_1.copy()
X = X.drop("movieId",axis = 1)
Kmeans = KMeans(n_clusters = cluster_num,init ="random",random_state = 0)
y = Kmeans.fit_predict(X)
#tagとrelevanceにratingを用いて重みづけをする。

for i in range(cluster_num):
    v = np.array(X[y == i])
    flag = ur_recommend["movieId"].isin(tag_rel_1[y==i]["movieId"])
    r = np.array(ur_recommend[flag]["rating"])
    weighted_tag =  cal_weighted_sum(v,r)
    euclid_rel = np.apply_along_axis(lambda x:dis.euclidean(weighted_tag,x),1,tag_rel)
    sort_euclid_rel = np.argsort(euclid_rel)
    print("クラスターNo.",i, "のおすすめ")
    for i in sort_euclid_rel[:5]:
        flag = movie_data["movieId"].isin([tag_id["movieId"][i*1128]])
        print(movie_data["title"][np.where(flag == True)[0][0]])
    print("###########################################")
print("以上が"+ ur_name + "さんへのおすすめです！是非ご覧ください！")